In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl as px
from sklearn.model_selection import train_test_split

## Data Pre-processing

In [18]:
AAPL = pd.read_excel("./data/AAPL_EQUITY.xlsx")
SPESG = pd.read_excel(('./data/SPESG_Index_5Y.xlsx')).dropna()

# Take SPESG and AAPL and combine them into one data frame: df['Date', 'SPESG', 'AAPL_Px', 'AAPL_Volume', 'AAPL_SMAVG(15)']
df = pd.merge(AAPL, SPESG, on='Date', how='inner')
new_columns = {'Last Px_x' : 'AAPL_Px', 'Volume': 'AAPL_Volume(M)', 'SMAVG(15)' : 'AAPL_SMAVG15(M)', 'Last Px_y': 'SPESG'}
df.rename(columns=new_columns, inplace=True)

# Strip M and convert these columns from object into float64
df['AAPL_Volume(M)'] = df['AAPL_Volume(M)'].str.strip('M').astype('float64')
df['AAPL_SMAVG15(M)'] = df['AAPL_SMAVG15(M)'].str.strip('M').astype('float64')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Quarter'] = df['Date'].dt.quarter
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.strftime('%A')

## Calculate returns

In [19]:
ret_period = [1, 5, 10, 30, 60, 120, 250]

for p in ret_period:
    df[f'AAPL_Px_ret{p}'] = ((df['AAPL_Px'] - df['AAPL_Px'].shift(-p)) / df['AAPL_Px'].shift(-p)).round(4)

df['AAPL_ret_f1'] = df['AAPL_Px_ret1'].shift(1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1274 entries, 0 to 1273
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             1274 non-null   datetime64[ns]
 1   AAPL_Px          1274 non-null   float64       
 2   AAPL_Volume(M)   1274 non-null   float64       
 3   AAPL_SMAVG15(M)  1260 non-null   float64       
 4   SPESG            1274 non-null   float64       
 5   Year             1274 non-null   int32         
 6   Quarter          1274 non-null   int32         
 7   Month            1274 non-null   int32         
 8   Day              1274 non-null   object        
 9   AAPL_Px_ret1     1273 non-null   float64       
 10  AAPL_Px_ret5     1269 non-null   float64       
 11  AAPL_Px_ret10    1264 non-null   float64       
 12  AAPL_Px_ret30    1244 non-null   float64       
 13  AAPL_Px_ret60    1214 non-null   float64       
 14  AAPL_Px_ret120   1154 non-null   float64

In [20]:
weekday_dummies = pd.get_dummies(df['Day']).astype(int)
df = pd.concat([df, weekday_dummies], axis=1)
df = df.dropna()
df.sort_values(by='Date', ascending=True, inplace=True)

X = df.drop(['Date', 'Day', 'Year', 'AAPL_ret_f1'], axis=1)
y = df['AAPL_ret_f1']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_train.head()

,AAPL_Px,AAPL_Volume(M),AAPL_SMAVG15(M),SPESG,Quarter,Month,AAPL_Px_ret1,AAPL_Px_ret5,AAPL_Px_ret10,AAPL_Px_ret30,AAPL_Px_ret60,AAPL_Px_ret120,AAPL_Px_ret250,Friday,Monday,Thursday,Tuesday,Wednesday
627,148.85,56.095,65.534,393.81,4,10,-0.0031,-0.0027,0.0563,-0.0012,0.0101,0.1432,0.2908,0,0,0,0,1
476,145.38,88.570,99.805,359.14,2,6,-0.0386,0.0111,0.0585,-0.1264,-0.1078,-0.1899,0.1471,1,0,0,0,0
981,81.28,104.491,123.868,264.90,2,6,0.0055,0.0220,0.0383,0.2115,0.2215,0.2110,0.7553,0,0,0,0,1
856,119.05,169.410,100.920,309.03,4,11,0.0211,0.0146,-0.0018,0.0003,-0.0151,0.3496,0.8354,0,1,0,0,0
607,161.94,69.464,76.870,409.75,4,11,0.0033,0.0551,0.0948,0.1492,0.0666,0.2863,0.3890,0,0,0,0,1


# Modeling
## Experimenting phase
### Linear Regression

In [22]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regressor = linear_model.LinearRegression()
regressor.fit(X_train, y_train)

# Training set
y_pred_train = regressor.predict(X_train)
r2_train = r2_score(y_train, y_pred_train)

# Test set
y_pred_test = regressor.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)

print(f"r2_train: {r2_train}")
print(f"r2_test: {r2_test}")

def adjusted_r2_score(r_squared, n, k):
    return 1 - ((1 - r_squared) * (n - 1) / (n - k - 1))

n_samples = X_train.shape[0]
n_features = X_train.shape[1]

adj_r2_train = adjusted_r2_score(r2_train, n_samples, n_features)
adj_r2 = adjusted_r2_score(r2_test, n_samples, n_features)
print("Adjusted R-squared on Training Data:", adj_r2_train)
print("Adjusted R-squared on Test Data:", adj_r2)

r2_train: 0.059757947119512544
r2_test: -0.089056775911168
Adjusted R-squared on Training Data: 0.02535884762388496
Adjusted R-squared on Test Data: -0.12890031649328404


### Bayesian Regression

In [23]:
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, r2_score

# Create Bayesian Ridge regression object
regressor = BayesianRidge()

# Fit the model
regressor.fit(X_train, y_train)

# Predict on training set
y_pred_train = regressor.predict(X_train)
r2_train = r2_score(y_train, y_pred_train)

# Predict on test set
y_pred_test = regressor.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)

print(f"r2_train: {r2_train}")
print(f"r2_test: {r2_test}")

# Function to calculate adjusted R^2
def adjusted_r2_score(r_squared, n, k):
    return 1 - ((1 - r_squared) * (n - 1) / (n - k - 1))

n_samples = X_train.shape[0]
n_features = X_train.shape[1]

adj_r2_train = adjusted_r2_score(r2_train, n_samples, n_features)
adj_r2_test = adjusted_r2_score(r2_test, n_samples, n_features)
print("Adjusted R-squared on Training Data:", adj_r2_train)
print("Adjusted R-squared on Test Data:", adj_r2_test)


r2_train: 0.014833057925065063
r2_test: 0.0005159829804658456
Adjusted R-squared on Training Data: -0.02120963507767648
Adjusted R-squared on Test Data: -0.03605050544707811


In [64]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Assuming y_true_train, y_pred_train, y_true_test, y_pred_test are your actual and predicted values
# for both the training and test sets

# Mean Absolute Error (MAE)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

# Mean Squared Error (MSE)
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

# Root Mean Squared Error (RMSE)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

# R2 Score
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print("MAE - Train: {:.4f}, Test: {:.4f}".format(mae_train, mae_test))
print("MSE - Train: {:.4f}, Test: {:.4f}".format(mse_train, mse_test))
print("RMSE - Train: {:.4f}, Test: {:.4f}".format(rmse_train, rmse_test))
print("R2 Score - Train: {:.4f}, Test: {:.4f}".format(r2_train, r2_test))

MAE - Train: 1.3939, Test: 1.4765
MSE - Train: 3.7658, Test: 4.5848
RMSE - Train: 1.9406, Test: 2.1412
R2 Score - Train: 0.0223, Test: 0.0120


In [60]:
X_test.head()

,AAPL_Volume(M),AAPL_SMAVG15(M),SPESG-1,Quarter,Month,AAPL_ret_perc-1,Friday,Monday,Thursday,Tuesday,Wednesday
1182,146.190,141.621,237.25,3,8,-2.976,0,0,1,0,0
232,50.747,57.523,367.36,2,5,-1.515,0,0,0,0,1
1215,83.599,96.118,243.01,2,6,-0.030,1,0,0,0,0
592,131.063,120.621,410.26,4,12,2.851,0,0,1,0,0
729,59.279,78.056,358.08,2,6,0.628,0,0,1,0,0


In [71]:
MSFT = pd.read_excel('data/MSFT_EQUITY.xlsx')
MSFT['Volume'] = MSFT['Volume'].str.strip('M').astype('float64').shift(-1)
MSFT['SMAVG(15)'] = MSFT['SMAVG(15)'].str.strip('M').astype('float64').shift(-1)
MSFT['Date'] = pd.to_datetime(MSFT['Date'])
MSFT['Year'] = MSFT['Date'].dt.year
MSFT['Quarter'] = MSFT['Date'].dt.quarter
MSFT['Month'] = MSFT['Date'].dt.month
MSFT['Day'] = MSFT['Date'].dt.strftime("%A")
df = pd.merge(MSFT, SPESG, on='Date', how='inner')
new_columns = {'Last Px_x': 'AAPL_Px', 'Last Px_y': 'SPESG', 'SMAVG(15)':'AAPL_SMAVG15(M)', 'Volume':'AAPL_Volume(M)'}
df.columns = df.columns.str.strip()
df.rename(columns=new_columns, inplace=True)

df['AAPL_Px_Shift-1'] = df['AAPL_Px'].shift(-1)
df['SPESG-1'] = df['SPESG'].shift(-1)

weekday_dummies = pd.get_dummies(df['Day']).astype(int)
df = pd.concat([df, weekday_dummies], axis=1)
df = df.dropna()
df.sort_values(by='Date', ascending=True, inplace=True)
X = df.drop(['Date', 'AAPL_Px', 'Day', 'SPESG'], axis=1)
y = df['AAPL_Px']

# y_pred = regressor.predict(X)

In [72]:
# Define the feature names in the same order as they were used during model training
feature_names = ['AAPL_Px_Shift-1', 'SPESG-1', 'AAPL_SMAVG15(M)', 'AAPL_Volume(M)', 'Year', 'Quarter', 'Month', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Create a new DataFrame with columns in the desired order
df_reordered = df[['Date', 'AAPL_Px', 'Day', 'SPESG'] + feature_names]

# Drop rows with NaN values
df_reordered = df_reordered.dropna()

# Sort the DataFrame by 'Date' column
df_reordered.sort_values(by='Date', ascending=True, inplace=True)

# Separate features (X) and target variable (y)
X_reordered = df_reordered.drop(['Date', 'AAPL_Px', 'Day', 'SPESG'], axis=1)
y_reordered = df_reordered['AAPL_Px']

# Make predictions using the reordered features
y_pred_reordered = regressor.predict(X_reordered)


ValueError: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.
